In [1]:
import tensorflow as tf
import numpy as np

/home/hojoon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/hojoon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Data

In [2]:
def one_hot(x):
    X = np.zeros([len(x), max(x)+1])
    X[np.arange(len(x)),x] = 1
    
    return X

In [3]:
x_data = np.array([[0,0], [1,0], [1,1], [0,0], [0,0], [0,1]])
y_data = np.array([0,1,2,0,0,2])
y_data = one_hot(y_data)

### Build Graph

In [4]:
X = tf.placeholder(tf.float32, shape = [None, 2])
Y = tf.placeholder(tf.float32, shape = [None, 3])

In [5]:
global_step = tf.Variable(0, trainable=False, name = 'global_step')

In [6]:
with tf.name_scope("layer1"):
    W1 = tf.Variable(tf.random_uniform([2,5],-1.0, 1.0))
    b1 = tf.Variable(tf.zeros([5]))
    
    h1 = tf.add(tf.matmul(X, W1),b1)
    a1 = tf.nn.relu(h1)
    
    tf.summary.histogram("W1",W1)

    
with tf.name_scope("layer2"):
    W2 = tf.Variable(tf.random_uniform([5,10],-1.0, 1.0))
    b2 = tf.Variable(tf.zeros([10]))
    
    h2 = tf.add(tf.matmul(a1, W2),b2)
    a2 = tf.nn.relu(h2)

    
with tf.name_scope("layer3"):
    W3 = tf.Variable(tf.random_uniform([10,3],-1.0, 1.0))
    b3 = tf.Variable(tf.zeros([3]))
    
    score = tf.add(tf.matmul(a2, W3),b3)

In [7]:
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = score))
    
# To track in the tensorboard
tf.summary.scalar('loss',loss)


with tf.name_scope("optimizier"):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
    train_op  = optimizer.minimize(loss, global_step= global_step)

In [8]:
with tf.name_scope("prediction"):
    pred   = tf.argmax(score, 1)
    target = tf.argmax(Y, 1)

    accuracy = tf.reduce_mean(tf.to_float(tf.equal(pred, target)))

In [9]:
merged = tf.summary.merge_all()

### Run Graph

In [10]:
with tf.Session() as sess:
    # Store all the variables into saver
    saver = tf.train.Saver(tf.global_variables())
    
    # Check Whether there exits a saved model
    ckpt  = tf.train.get_checkpoint_state('./model')
    # Restore the variables if there exists a saved model 
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
    
    # Allocate the directory to save the graph
    writer = tf.summary.FileWriter('./logs', sess.graph)
    
    # Train
    for step in range(100):
        sess.run(train_op, feed_dict = {X:x_data, Y:y_data})
        
        print('Step:%d  ' % sess.run(global_step),
              'Loss:%.3f' % sess.run(loss, feed_dict = {X:x_data, Y:y_data}))
    
        # Save the summary
        summary = sess.run(merged, feed_dict = {X:x_data, Y:y_data})
        writer.add_summary(summary, global_step = sess.run(global_step))
    writer.close()
    
    # Save the model
    saver.save(sess, './model/dnn.ckpt', global_step = global_step)
    
    
    # Prediction
    print('\nPrediction:', sess.run(pred, feed_dict = {X:x_data, Y:y_data}))
    print('True:      ', np.argmax(y_data,1))
    print('Accuracy: %.2f' %sess.run(accuracy, feed_dict = {X:x_data, Y:y_data}))
    

Step:1   Loss:1.593
Step:2   Loss:1.492
Step:3   Loss:1.404
Step:4   Loss:1.328
Step:5   Loss:1.260
Step:6   Loss:1.198
Step:7   Loss:1.143
Step:8   Loss:1.095
Step:9   Loss:1.056
Step:10   Loss:1.021
Step:11   Loss:0.988
Step:12   Loss:0.960
Step:13   Loss:0.935
Step:14   Loss:0.910
Step:15   Loss:0.886
Step:16   Loss:0.863
Step:17   Loss:0.841
Step:18   Loss:0.820
Step:19   Loss:0.800
Step:20   Loss:0.781
Step:21   Loss:0.763
Step:22   Loss:0.745
Step:23   Loss:0.731
Step:24   Loss:0.716
Step:25   Loss:0.701
Step:26   Loss:0.686
Step:27   Loss:0.671
Step:28   Loss:0.655
Step:29   Loss:0.639
Step:30   Loss:0.624
Step:31   Loss:0.607
Step:32   Loss:0.593
Step:33   Loss:0.577
Step:34   Loss:0.562
Step:35   Loss:0.548
Step:36   Loss:0.534
Step:37   Loss:0.521
Step:38   Loss:0.508
Step:39   Loss:0.494
Step:40   Loss:0.482
Step:41   Loss:0.469
Step:42   Loss:0.455
Step:43   Loss:0.443
Step:44   Loss:0.431
Step:45   Loss:0.419
Step:46   Loss:0.407
Step:47   Loss:0.396
Step:48   Loss:0.385
S